In [ ]:
# ===== IMPORTACION (EJECUTAR SIEMPRE) ===== #
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from time import sleep

# ME TRAIGO MI CLASE
from AxesorMining import AxesorMining

#### OBTENCION DE TODAS LAS EMPRESAS DE AXESOR.ES
Ideas generales:
- Accedo a https://www.axesor.es/directorio-informacion-empresas<br>

1. PRIMER BUCLE: Por provincia
    - Accedo una a una a todas las provincias. **Todas en la misma URL.**
    <br>Formato: https://www.axesor.es/directorio-informacion-empresas/empresas-de-Alava

    <br>

2. SEGUNDO BUCLE: Por municipio
    - Accedo una a una a todos los municipios que hay en la página de provincia. **Aquí ya hay paginación.**
    <br>Formato: https://www.axesor.es/directorio-informacion-empresas/empresas-de-Alava/informacion-empresas-de-Alegria-Dulantzi/1
        - Recorrer todos los valores de la página.
        - Saltar a la siguiente página.
        - Si es la última página, paso a siguiente municipio.
        - Si es el último municipio, paso a la siguiente provincia.

    <br>

- Creo un objeto donde voy a ir almacenando.
- Inserto los valores
- Genero un CSV




-------------

##### OBTENGO LA LISTA DE PROVINCIAS

In [ ]:
try:

    # PUNTO DE PARTIDA:
    url_base = 'https://www.axesor.es/directorio-informacion-empresas'

    # LA OBTENGO
    soup = AxesorMining().request_soup(url_base)    

    # APUNTO A LOS LINKS DE LAS PROVINCIAS
    provincias_table = soup.find(id="bloque_listadoProvincias").find('table', class_='provincias')
    a_provincias = provincias_table.find_all('a')

    # DICCIONARIO CON LAS URL DE PROVINCIAS
    dict_url_provincias = {link.get_text(strip=True): 'http:' + link.get('href') for link in a_provincias}

    # REORDENO ALFABETICAMENTE
    dict_url_provincias = dict(sorted(dict_url_provincias.items()))

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))


OBTENGO LOS MUNICIPIOS

In [ ]:
try:

    # PRIMER NIVEL DE BUCLE: PROVINCIAS
    for provincia, url_provincia in dict_url_provincias.items():

        # SACO EL NOMBRE DE LA PROVINCIA
        print('Obteniendo municipios de ' + provincia)

        # OBTENGO EL CONTENIDO
        soup = AxesorMining().request_soup(url_provincia)   

        # APUNTO A LOS LINKS DE LOS MUNICIPIOS
        a_municipios = soup.find(id="listaMunicipiosProvincia").find('table', class_='provincias').find_all('a')

        # LISTA DE URLS DE MUNICIPIOS DE LA PROVINCIA
        listado_url_municipios = [[provincia, link.get_text(strip=True), 'http:' + link.get('href')] for link in a_municipios]

        # PASO LA BBDD A DATAFRAME
        df = pd.DataFrame(listado_url_municipios, columns=['Provincia', 'Municipio', 'URL'])
        df.sort_values(by=['Municipio'], ascending=True, inplace=True)
        nombre_csv = f"data/0_Provincias/{provincia}.csv"

        # GENERO EL CSV
        df.to_csv(nombre_csv, sep=';', index=False)
        print('Generado con éxito -> ' + nombre_csv)
        print('------------------')

    print('---------- PROCESO FINALIZADO CON ÉXITO ----------')

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))
